## 爬取策略

1. 爬虫分为2步，2个步骤分开执行

   + 抓取每个城市的所有详情页url，保存在{城市：详情页url列表}的字典里，存储为json文件供下一步抓取。
   + 遍历每个城市的所有详情页url，抓取购物中心名称、评分、人均价格等信息

2. 第1步爬虫可以在同一个浏览器session中进行，这样只需要登录一次。不需要每次爬取一个城市登录一次。

3. 需要维护一份{城市名称:城市名称缩写}的字典供第1步遍历用，如上海 <https://sh.meituan.com/s/购物中心>

4. 但要注意名称冲突，如苏州和深圳，二者缩写相同

In [10]:
import json
import os
import time
from tqdm import tqdm
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [4]:
# 打开一个浏览器session，打开美团网页手动登录(千万不要关闭)，之后在这个页面中完成所有爬取
bro = webdriver.Chrome()
bro.get('https://bj.meituan.com/s/')

In [8]:
def oneCityurl(city):
    '''得到一个城市的url'''
    url = "https://" + city + ".meituan.com/s/购物中心/"
    return url

def getOnepage(browser, url):
    '''爬取一个网页的内容'''
    browser.get(url)
    time.sleep(3)
    return browser, browser.page_source

def getDetailurl(cityPagesource):
    '''从页面解析章节标题、详情页url'''
    soup=BeautifulSoup(cityPagesource,'lxml',fromEncoding='ISO-8859-1')
    div_list=soup.select('#react > div > div > div.center-content.clearfix > div.left-content > div.common-list > div.common-list-main > div')
    urlList = []
    for div in div_list:
        detail_url='https:'+div.a['href']
        urlList.append(detail_url)
    return urlList

def flipPages(aURL):
    '''对每一个城市的详情页翻页循环爬取'''
    urlist = []
    count = 0
    browser, pg = getOnepage(bro, aURL)
    while True:
        count += 1
        print("正在抓取第{}个页面".format(count))
        time.sleep(3)
        try:
            onpageurl = getDetailurl(browser.page_source)
        except:
            print("已翻到最后一页")
            break
        urlist.extend(onpageurl)
        try:
            next_btn = browser.find_element_by_xpath(
                        '//*[@id="react"]/div/div/div[2]/div[1]/nav/ul/li[7]/a')
            next_btn.click()
        except:
            print("翻页失败，跳到下一页")
            continue
        if count>50:
            break
    return list(set(urlist))

def getOnedetail(url, bro, city='null'):
    '''输入一个详情页url，输出详细信息dict，共7个字段
       没有抓取成功的页面全部返回null
       
    '''
    try:
        bro.get(url)
    except:
        print('详情页爬取失败')
        plazaDict1 = {'city':'null', # 城市
                 'name': 'null', #购物中心名称
                'score':'null', #评分
             'avgPrice':'null', #平均价格
              'address':'null', #地址
                  'lng':'null', #经度
                  'lat':'null', #纬度
                'title':'null'}
        return plazaDict1
    pageText = bro.page_source
    detail_soup = BeautifulSoup(pageText, 'lxml', fromEncoding='ISO-8859-1')
    try:
        script=detail_soup.find_all('script')[10]
    except: 
        script='null'
    try:
        name=str(script).split(',')[3].split(':')[1].replace("\""," ")
        name=name.replace(' ' , '')
    except: 
        name='null'
    try:
        score=str(script).split(',')[4].split(':')[1].replace("\""," ")
    except: 
        score='null'
    try:
        avgPrice=str(script).split(',')[5].split(':')[1].replace("\""," ")
    except:
        avgPrice='null'
    try:
        address=str(script).split(',')[6].split(':')[1].replace("\""," ")
        address=address.replace(' ' , '')
    except:
        address='null'
    try:
        lng=str(script).split(',')[12].split(':')[1].replace("\""," ")
    except:
        lng='null'
    try:
        lat=str(script).split(',')[13].split(':')[1].replace("\""," ")
    except:
        lat='null'
    try:
        title=str(script).split(',')[20].split(':')[1].replace("\""," ")
        title=title.replace(' ' , '')
    except:
        title='null'
    plazaDict = {'city':city, # 城市
                 'name':name, #购物中心名称
                'score':score, #评分
             'avgPrice':avgPrice, #平均价格
              'address':address, #地址
                  'lng':lng, #经度
                  'lat':lat, #纬度
                'title':title} #类型
    return plazaDict

In [8]:
# 定义需要爬取的所有城市和简称的dict
cityList = {
   "北京":"bj",
   "上海":"sh",
   "广州":"gz",
   "鞍山":"as"
}

if __name__=='__main__': # 主程序入口

    cityUrls = {} # 创建城市名称为key，value为购物中心详情页列表的dict
    for k, v in cityList.items():
        print(k, oneCityurl(v))
        url = oneCityurl(v)
        cityurlist = flipPages(url)
        cityUrls[k] = cityurlist

    if os.path.exists("./cityUrls.json"):
        os.remove("./cityUrls.json")
    with open("./cityUrls.json","w") as f:
        json.dump(cityUrls,f) 
    print("将所有详情页url存入json文件")

北京 https://bj.meituan.com/s/购物中心/
正在抓取第1个页面
正在抓取第2个页面
正在抓取第3个页面
正在抓取第4个页面
正在抓取第5个页面
正在抓取第6个页面
正在抓取第7个页面
正在抓取第8个页面
正在抓取第9个页面
正在抓取第10个页面
正在抓取第11个页面
正在抓取第12个页面
正在抓取第13个页面
正在抓取第14个页面
正在抓取第15个页面
正在抓取第16个页面
正在抓取第17个页面
正在抓取第18个页面
正在抓取第19个页面
正在抓取第20个页面
正在抓取第21个页面
正在抓取第22个页面
正在抓取第23个页面
正在抓取第24个页面
正在抓取第25个页面
正在抓取第26个页面
正在抓取第27个页面
正在抓取第28个页面
正在抓取第29个页面
正在抓取第30个页面
正在抓取第31个页面
正在抓取第32个页面
正在抓取第33个页面
正在抓取第34个页面
正在抓取第35个页面
正在抓取第36个页面
正在抓取第37个页面
正在抓取第38个页面
正在抓取第39个页面
正在抓取第40个页面
正在抓取第41个页面
正在抓取第42个页面
正在抓取第43个页面
正在抓取第44个页面
正在抓取第45个页面
正在抓取第46个页面
正在抓取第47个页面
正在抓取第48个页面
正在抓取第49个页面
正在抓取第50个页面
正在抓取第51个页面
上海 https://sh.meituan.com/s/购物中心/
正在抓取第1个页面
正在抓取第2个页面
正在抓取第3个页面
正在抓取第4个页面
正在抓取第5个页面
正在抓取第6个页面
正在抓取第7个页面
正在抓取第8个页面
正在抓取第9个页面
正在抓取第10个页面
正在抓取第11个页面
正在抓取第12个页面
正在抓取第13个页面
正在抓取第14个页面
正在抓取第15个页面
正在抓取第16个页面
正在抓取第17个页面
正在抓取第18个页面
正在抓取第19个页面
正在抓取第20个页面
正在抓取第21个页面
正在抓取第22个页面
正在抓取第23个页面
正在抓取第24个页面
正在抓取第25个页面
正在抓取第26个页面
正在抓取第27个页面
正在抓取第28个页面
正在抓取第29个页面
正在抓取第30个页面
正在抓取第31个页面
正在抓取第32个页面
正在抓取第33个页面
正在抓取第34个页面
正在抓取第35个页面
正在抓取

In [14]:
# 爬详情页部分代码
if __name__=='__main__':
    with open("./cityUrls.json",'r') as r:
        cityUrls = json.load(r)
    start = time.time()
    cityData = pd.DataFrame()
    for city, urlist in cityUrls.items():
        print('\n正在处理:', city)
        for url in tqdm(urlist):
            time.sleep(1.5)
            oneRow = getOnedetail(url, bro, city)
            cityData = cityData.append(oneRow, ignore_index=True)
    print('\n程序耗时{:.2f}秒'.format(time.time()-start))
    cityData.to_csv('./city_shop.csv', encoding="utf_8_sig", index=False)

  0%|          | 0/32 [00:00<?, ?it/s]
正在处理: 北京
  0%|          | 0/32 [00:00<?, ?it/s]
正在处理: 上海
  0%|          | 0/32 [00:00<?, ?it/s]
正在处理: 广州
  0%|          | 0/32 [00:00<?, ?it/s]
正在处理: 鞍山
100%|██████████| 32/32 [01:39<00:00,  3.11s/it]
程序耗时406.24秒



In [15]:
cityData

,address,avgPrice,city,lat,lng,name,score,title
0,海淀区巴沟路2号（近地铁10号线巴沟站）,200,北京,39.973102,116.295109,北京华联万柳购物中心,4.5,商场
1,石景山区阜石路300号（近苹果园）,195,北京,39.922944,116.1717,喜隆多新国际购物中心,5,商场
2,朝阳区北四环东路108号华堂商场一层,203,北京,39.987983,116.421017,伊藤洋华堂（亚运村店）,5,商场
3,丰台区丰台街道丰台北路18号,184,北京,39.866335,116.306145,金唐购物中心,4.6,商场
4,海淀区农大南路1号1号楼（近上地南口）,279,北京,40.028531,116.311311,北京华联上地购物中心,4.5,商场
...,...,...,...,...,...,...,...,...
123,立山区鞍千路250号,0,鞍山,41.145167,123.055349,百姓新生活购物中心,3.5,商场
124,铁东区铁东二道街95甲（景子街西座）,0,鞍山,41.113693,122.991579,新都购物广场,3.5,商场
125,null,null,鞍山,null,null,null,null,null
126,海城市海州大街48号,0,鞍山,40.860953,122.747456,普临购物广场,3,商场
